# Bagging and Random Forests

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as scs

from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

from sklearn.datasets import load_breast_cancer, load_iris
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

## Objectives

Morning Objectives

 * Explain the relationship and difference between bagging and a random forest.
 * Explain why bagging/random forests are more accurate than a single decision tree.
 * Explain & construct a random forest (classification or regression).

Afternoon Objectives

 * Get feature importances from a random forest.
 * Explain how OOB error is calculated and what it estimates.

## Agenda

Morning Agenda:

 * Discuss ensemble methods
 * Review bias/variance tradeoff
 * Review decision trees
 * Discuss bagging (bootstrap aggregation)
 * Discuss random forests

Afternoon Agenda:

 * Discuss out-of-bag error
 * Discuss feature importance



## What is an Ensemble Method?

In general, an **ensemble** method combines many weak models to form a strong model. We train multiple *different* models on the data. They could be trained on different subsets of the data, or trained in different ways, or even be completely different types of models.

Once we've done that, we need to combine the models to form a single model.

Class discussion: how would get a single prediction from an ensemble of **regression** models?

Class discussion: how would get a single prediction from an ensemble of **classification** models?



## Ensembles: Intuition

Suppose we have 5 *independent* hard binary classifers (they only give 0 or 1 probability). If each has a %70 probability of correctly classifying a data point, what is the probability that an *ensemble* of these models correctly classifies a point? The ensemble prediction is just the majority vote of the individual classifiers.

Question: what does independent mean?

In [ ]:
def find_ensemble_probability(n, p):
    '''Given a n independent classifiers each with
    probability p of voting correctly, return the 
    probability that the majority of the classifiers 
    vote correctly'''
    return (1 - scs.distributions.binom(n,p).cdf(n//2))

In [ ]:
find_ensemble_probability(5, 0.7)

In [ ]:
ns = np.arange(1, 55, 2)
ensemble_accuracies = [find_ensemble_probability(n, 0.7) for n in ns]

fig, ax = plt.subplots()
ax.plot(ns, ensemble_accuracies, '.')
ax.set_ylabel("Ensemble accuracy")
ax.set_xlabel("Number of independent 0.7-accuracy classifiers")
ax.set_title("Accuracy of an Ensemble of Independent Classifiers");

$$ \binom{5}{5} 0.7^5 + \binom{5}{4} 0.7^4 0.3 + \binom{5}{3} 0.7^3 0.3^2 \approx 0.83 $$

With 55 such classifiers we can achieve 99.9% accuracy.


Question: what's the limitation?

## How to Make Them Independent?

If the learners are all the same, ensembles don't help.

Train each learner on different subset of data.

Question: Why is this better than a single good model?

## Bias and Variance

Question: what is bias?

Question: what is variance?

Question: what is the bias of an unpruned decision tree?

## Review: Classification Trees

A **classification tree** is a decision tree to predicts whether a data point is in one class or another. Each branch node is a decision, choosing left or right based on the value of a certain feature. Each leaf node gives the probability that a data point is in one class or another.

Let's look at the tennis dataset from the other day.

In [ ]:
# Read in our data
tennis_df = pd.read_table('data/tennis.txt', delim_whitespace=True)
tennis_df.rename(columns={'playtennis': 'played'}, inplace=True)
#tennis_df['played'] = tennis_df['played'].apply(lambda x: 1 if x == 'yes' else 0)
tennis_df

You probably need to do `pip install graphviz` to generate this.

In [ ]:
from graphviz import Digraph
dot = Digraph(comment='A simple classification tree')

dot.node('O', 'outlook?', shape='diamond')
dot.node('1', "no", shape='rectangle')
dot.node('H', 'humidity?', shape='diamond')
dot.node('O2', 'outlook?', shape='diamond')
dot.node('W', 'wind?', shape='diamond')
dot.node('3', 'yes', shape='rectangle')
dot.node('T', 'temperature?', shape='diamond')
dot.node('4', 'yes', shape='rectangle')
dot.node('5', "no", shape='rectangle')
dot.node('2', "no", shape='rectangle')
dot.node('W2', 'wind?', shape='diamond')
dot.node('6', "no", shape='rectangle')
dot.node('7', 'yes', shape='rectangle')

dot.edge('O', '1', 'overcast')
dot.edge('O', 'H', 'not overcast')
dot.edge('H', 'O2', 'high')
dot.edge('H', 'W', 'normal')
dot.edge('W', '3', 'False')
dot.edge('W', 'T', 'True')
dot.edge('T', '4', 'mild')
dot.edge('T', '5', 'cool')
dot.edge('O2', '2', 'sunny')
dot.edge('O2', 'W2', 'rainy')
dot.edge('W2', '7', 'False')
dot.edge('W2', '6', 'True')
dot

A classification tree is built by

* Iteratively divide the nodes such that the information gain (using entropy/gini impurity) is maximized
* Various stopping conditions like a depth limit
* Prune trees by merging nodes

## Review: Regression Trees

A **regression tree** predicting a number rather than a class.  A leaf node outputs *the mean of its elements* instead of *the majority class of its elements*.

To train a regression tree, we

* Iteratively divide the nodes such that *variance* of the parent node $P$ minus the weighted variance of the child nodes $C_L$ and $C_R$ is maximized

$$I_V = \text{var}(P) -\frac{|C_R|}{|P|}\text{var}(C_L) - \frac{|C_R|}{|P|}\text{var}(C_R) $$


$$I_V = \frac{1}{|P|}\left(\sum_{i \in P} (y_i - \bar{y_P})^2\right) - \left(\frac{|C_L|}{|P|}\frac{1}{|C_L|}\sum_{i \in C_L} (y_i - \bar{y_L})^2 + \frac{|C_R|}{|P|}\frac{1}{|C_R|}\sum_{i\in C_R} (y_i - \bar{y_R})^2\right)$$

$$I_V = \frac{1}{|P|}\left[\left(\sum_{i \in P} (y_i - \bar{y_P})^2\right) - \left(\sum_{i \in C_L} (y_i - \bar{y_L})^2 + \sum_{i\in C_R} (y_i - \bar{y_R})^2\right)\right]$$

* Use various stopping conditions like a depth limit, minimum leaf size, and
* Prune trees by merging nodes

## Regression Trees: Example

 $x_1$ |   $x_2$ |  $y$
-------|---------|--------
 1     |    1    |   1
 0     |    0    |   2
 1     |    0    |   3
 0     |    1    |   4

 Prior to the split we guess the mean, 2.5, for everything, giving total squared error:
 
 $$ E = (1-2.5)^2 + (2-2.5)^2 + (3-2.5)^2 + (4-2.5)^2  = 5$$
 After we split on $x_1$ we guess 2 for rows 1 & 3 and 3 for rows 2 & 4:
 
 $$ I = 5 - [(1-2)^2 + (3-2)^2 + (2-3)^2 + (4-3)^2] = 5 - 4  = 1 $$

## Decision Tree Summary

What are the pros and cons?

Pros
 * No feature scaling needed
 * Model nonlinear relationships
 * Can do both classification and regression
 * Robust
 * Highly interpretable

Cons
 * Can be expensive to train
 * Need to be tuned so they don't totally overfit


## Review: Bootstrapping

What is a bootstrap sample?

What have we learned that bootstrap samples are good for so far?

Let's get the median of some data.

In [ ]:
data = scs.uniform(0,10).rvs(100)
np.median(data)

What's the confidence interval of this estimate?

In [ ]:
def bootstrap_confidence_interval(data, function, alpha=0.05, n_bootstraps=1000):
    '''return a the confidence interval for a function of data using bootstrapping'''
    medians = []
    for _ in range(n_bootstraps):
        bootstrap_sample = np.random.choice(data, len(data))
        medians.append(function(bootstrap_sample))
    return (np.percentile(medians, 100*(alpha/2.)),
            np.percentile(medians, 100*(1-alpha/2.)))

In [ ]:
alpha = .05
ci = bootstrap_confidence_interval(data, np.median, alpha)

print("The {}% confidence interval is from {} to {}".format(1-alpha, *ci))

Our procedure was
  * Take 1000 bootstrap samples.
  * Take the median of each sample.
  * The 95% confidence inverval for the median is between the 25th and 975th largest samples.
  
To confirm this worked, let's do it a bunch of times.

In [ ]:
hits = []
for _ in range(100):
    data = scs.uniform(0,10).rvs(100)
    ci = bootstrap_confidence_interval(data, np.median)
    hits.append(ci[0] < 5.0 and ci[1] > 5.0)
print(np.mean(hits))

This is the fraction of time the bootstrapped CI contained the true median

## Bootstrapped aggregation

 * We are thinking about the population of all possible decision tree models on our data.
 * If I take $n$ samples *iid* from this distribution and average them the variance goes down by $\sqrt n$
 * There is some correlation between models because they are all trained on bootstrap samples from the same draw.

## An Experiment

You're each going to be a decision tree on some data based on a bootstrap sample, and then we'll all ensemble the results.

In [ ]:
data = load_iris()

# Split into test/train, using the same random state for everyone
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, random_state=462)

In [ ]:
clf = DecisionTreeClassifier()
# each of you has a different bootstrap sample
indices = np.arange(len(X_train))
bootstrap_sample_indices = np.random.choice(indices, len(X_train))
clf.fit(X_train[bootstrap_sample_indices], y_train[bootstrap_sample_indices])

In [ ]:
print("Accuracy = {:.3f}".format(np.mean(clf.predict(X_test) == y_test)))

In [ ]:
print("My prediction: {}".format(clf.predict(X_test)[0:20]))
print("Actual result: {}".format(y_test[0:20]))

What is your prediction?

This concept, creating each model from a bootstrap sample and aggregating the results, is called **bagging**. It can be used with any sort of model, but is generally done with decision trees.

Question: why?

### Bootstrapping on Regression Trees

Let's do another example. Consider some points.

In [ ]:
n_data = 5000
x = scs.uniform(0, 10).rvs(n_data)
y = np.sin(x) + scs.norm(0, 0.5).rvs(n_data)

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
ax.plot(x, y, '.', ms=2);

We'll create a decision tree to predict the points. We won't make it that deep because we want to visualize the prediction.

In [ ]:
max_depth = 7
model0 = DecisionTreeRegressor(max_depth=max_depth)
model0.fit(x.reshape(-1, 1), y)

In [ ]:
n_pts = 1000
xpts = np.linspace(0, 10, n_pts)

fig, ax = plt.subplots(figsize=(15,6))
ax.plot(x, y, '.', ms=2)

ax.plot(xpts, model0.predict(xpts.reshape(-1, 1)), 'b');

That's what one decision tree looks like; each discontinuity is a node split. What if we creat a bunch of bootstrap samples and build an ensemble of trees from them?

In [ ]:
n_bootstraps = 5000
yptses = np.zeros((n_bootstraps, n_pts))
for i in range(n_bootstraps):
    bootstrap = np.random.choice(np.arange(n_data), n_data, replace=True)
    model = DecisionTreeRegressor(max_depth=max_depth)
    model.fit(x[bootstrap].reshape(-1, 1), y[bootstrap])
    yptses[i] = model.predict(xpts.reshape(-1, 1))


Here are three of the bootstrapped trees:

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
ax.plot(x, y, '.', ms=2)

for i in range(3):
    ax.plot(xpts, yptses[i])
;

And here in black is the average across all bootstrapped trees:

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
ax.plot(x, y, '.', ms=2)

ax.plot(xpts, yptses.mean(axis=0), 'k');

## Random Forests

Bagging decision trees is pretty cool, but the trees still tend to look pretty similar. We want a way to make the trees more different (decorrelate them) without substantially increasing the bias of each tree.

Random forests do this with **feature subsetting** (also called "subspace sampling"). For any node in any tree, instead of finding the best split among **all** features, we randomly select a few features, and find the best split among just those features. The number of features $m$ to consider at each split is a hyperparameter; typically $m = \sqrt k$ is used.

Again, the features to consider are chosen **at each split**, not each tree. **Everyone gets this wrong.**

For example, suppose we're building a model with nine features. One of them is really predictive, another is pretty good, and the others are just ok.

If we build an ensemble of bagged trees, probably each will use the good feature as the first split, and probably each will use the pretty-good feature at the next split. For the other splits the trees might differ, particularly farther down when only a few points are being considered, but the first branches will be pretty much the same.

If we build a tree in a random forest, we'll only consider three (random) features for that first split. Only a fraction of the trees (around 30%) will consider the good feature on the first split, so they will use that. Some of the others will consider the pretty-good feature, so they will start there. The others will start at some other feature. Those trees will still consider the good and pretty-good features at some of the lower nodes, and will get to take advantage of them, but the overall structure of those trees will be completely different.

## Random Forest Parameters

Random Forest Parameters

 * Total number of trees
 * Number of features to use at each split
 * Individual decision tree Parameters
    - e.g., tree depth, pruning, split criterion

In general, RF are fairly robust to the choice of parameters and overfitting.

## Pros and Cons of Random Forest

Pros

 * Often give near state-of-the-art performance
 * Good out-of-the-box performance
 * No feature scaling needed
 * Model nonlinear relationships

Cons

 * Can be expensive to train (though can be done in parallel)
 * Not interpretable

## Accuracy

Let's investigate the accuracy of a random forests compared with a single decision tree using the breast cancer dataset. 

In [ ]:
# Load data
data = load_breast_cancer()
#data = pd.drop(data, )
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

X = X.drop(['worst concave points', 'mean concave points', 'worst perimeter', 'worst radius', 'worst area', 'mean concavity'], axis=1)
# Split into test/train
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=.33,
                                                    random_state=0)


In [ ]:
X_train.columns

In [ ]:
y_train[:10]

First, consider a decision tree, doing a grid search over hyperparameters.

In [ ]:
# Parameter Search                                     
model = DecisionTreeClassifier()
depth_parm = np.arange(1, 12, 1)
num_samples_parm = np.arange(5,95,10)
parameters = {'max_depth' : depth_parm,
             'min_samples_leaf' : num_samples_parm}
clf = GridSearchCV(model, parameters, cv=10)
clf.fit(X_train,y_train)
print(clf.score(X_test, y_test))

In [ ]:
clf.best_estimator_

In [ ]:
clf.best_params_

Now random forests.

In [ ]:

# Train and fit model                                                   
rf = RandomForestClassifier(n_estimators=1000,
                           max_features='auto',
                           random_state=0)
rf.fit(X_train, y_train)
                                     
# Test Prediction
pred = rf.predict(X_test)
print(rf.score(X_test, y_test))

So that's better.

# Afternoon Lecture

## Interpreting Random Forests

## Objectives

Morning Objectives

 * Explain & construct a random forest (classification or regression).
 * Explain the relationship and difference between random forest and bagging.
 * Explain why random forests are more accurate than a single decision tree.

Afternoon Objectives

 * Get feature importances from a random forest.
 * Explain how OOB error is calculated and what is it an estimate of.

## Agenda

Morning Agenda

 * Discuss ensemble methods
 * Review bias/variance tradeoff
 * Review decision trees
 * Discuss bagging (bootstrap aggregation)
 * Discuss random forests

Afternoon Agenda

 * Discuss out-of-bag error
 * Discuss feature importance

## Review: Bagging and Random Forests

What is bagging?

Can bagging be used with other models?

What's the difference between bagging and random forests?

## Out-Of-Bag Error

The out-of-bag error is a way to measure the error of a bagged model (including random forests).

Since the decision trees are constructed from a bootstrapped sample, each tree will (probably) not see all of the data, so each data point will (probably) not be seen by many of the trees.

Let's imagine a data set with 10 points, and that we're making a random forest with 20 trees. We'll then construct 20 bootstrap samples.

In [ ]:
fake_data = np.arange(10)

n_data = len(fake_data)
n_trees = 20

np.random.seed(42)

bootstrap_samples = np.random.choice(np.arange(n_data),
                                     [n_trees, n_data],
                                     replace=True)
# sort along rows to make it a bit easier to read
bootstrap_samples.sort(axis=1)
bootstrap_samples

We would then construct 20 trees, each using the data in one of the rows.

Notice the first tree (and several others) isn't constructed using point 0 at all. So we could test that point of an ensemble of just those trees a get a independent measure of the effectiveness of the model. Simililarly, we could test other points on other trees.

Question: which trees would we use to test point 1?



In [ ]:
bootstrap_samples[(bootstrap_samples != 1).all(axis=1)]

On average, how many trees can we use to test each point?

In [ ]:
bsize = []
for i in range(10):
    bsize.append(sum((bootstrap_samples != i).all(axis=1)))

In [ ]:
np.mean(bsize)/n_trees

It turns out this number is $\frac{1}{e}$; you can [read about the math](https://stats.stackexchange.com/questions/88980/why-on-average-does-each-bootstrap-sample-contain-roughly-two-thirds-of-observat) and, relatedly, [derangements](https://en.wikipedia.org/wiki/Derangement)

Often we use cross validation anyway because we're comparing with other models and we want to use the same validation pipeline for everything.



## Feature Importances

One of the challenges of random forests is the lack of interpretability. Feature importances are a measure of which features aften actually effect the predictions.

This can be a critical business question. For example, with churn analysis, it's generally more important to understand *why* customers are churning than to predict which customers are going to churn.

How should we measure it?

## Feature Importances: Average Decrease in Impurity

How much does each feature decrease the impurity?

 * For each tree, each split is made in order to reduce the total impurity of the tree (Gini/entropy/MSE). 
 * "Total information gain" for a single tree is the sum of the information gain at each split, and a feature's importance for a single tree is the total information gain of splits involving that feature divided by the total information gain of the tree.
 * This is called "Gini importance" and it is what `sklearn` implements. 
 * The importance of a feature in a forest is the average of the Gini importances for that feature across all trees. 

In [ ]:
# Load data
data = load_breast_cancer()
#data = pd.drop(data, )
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

X = X.drop(['worst concave points', 'mean concave points', 'worst perimeter', 'worst radius', 'worst area', 'mean concavity'], axis=1)
# Split into test/train
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=.33,
                                                    random_state=0)


In [ ]:
rf = RandomForestClassifier(n_estimators=1000,
                           max_features='auto',
                           random_state=0)
rf.fit(X_train, y_train)

In [ ]:
# Plot the feature importance
feat_scores = pd.DataFrame({'Average Gini importance' : rf.feature_importances_},
                           index=X.columns)
feat_scores = feat_scores.sort_values(by='Average Gini importance')
feat_scores.plot(kind='barh');

## Feature Importances: Average Decrease in Accuracy when a feature's values are shuffled

(Described [here](https://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm#varimp))

How much does randomly shuffling the values of a feature affect accuracy?

To compute the importance of the $j^{th}$ feature:

 * When the $b^{th}$ tree is grown, use it to predict the OOB samples and record accuracy.
 * Scramble the values of the $j^{th}$ feature in the OOB samples and do the prediction again.  Compute the new (lower) accuracy.
 * Average the decrease in accuracy across all trees.

In [ ]:
# Load data
data = load_breast_cancer()
#data = pd.drop(data, )
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

X = X.drop(['worst concave points', 'mean concave points', 'worst perimeter', 'worst radius', 'worst area', 'mean concavity'], axis=1)

feature_names = X.columns
X = X.values
y = y.values
# Split into test/train
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=.33,
                                                    random_state=0)


In [ ]:
n_trees = 250
index_set = set(range(len(X)))
bootstrapped_indices = [np.random.choice(np.arange(len(X)), size=len(X)) for _ in range(n_trees)]
oob_indices = [list(index_set - set(b)) for b in bootstrapped_indices]
estimators = [DecisionTreeClassifier().fit(X[b],y[b]) for b in bootstrapped_indices]

In [ ]:
oob_scores = np.array([est.score(X[oob], y[oob]) for oob, est in zip(oob_indices, estimators)])

In [ ]:
def shuffle_column(X, i):
    X_new = X.copy()
    np.random.shuffle(X_new[:,i])
    return X_new

In [ ]:
feat_importance = np.zeros(X.shape[1])
for i in range(X.shape[1]):
    X_shuffled = shuffle_column(X, i)
    new_oob_scores = np.array([est.score(X_shuffled[oob], y[oob]) for oob, est in zip(oob_indices, estimators)])
    feat_importance[i] = (oob_scores - new_oob_scores).mean()

In [ ]:
importance_series = pd.Series(data=feat_importance, index=feature_names)
importance_series.sort_values().plot(kind='barh');